In [ ]:
import tkinter as tk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np
from scipy import interpolate
from scipy.integrate import odeint

In [ ]:
def model(y, t, params):

    b = 1000  # (dissipation-friction coefficient) parameter taken from the paper
    d = 10**8  # (nonlinear dissipation coefficient for bounded motion) parameter taken from the paper
    # Unpack the state variables
    x, v = y
    
    #params
    p,k= zip(*params)
    pt = np.interp(t, t, p)
    kt = np.interp(t, t, k)

    # Define the derivatives
    dxdt = v
    dvdt = -(pt - b) * v - kt * x - d * (x**2) * v
    
    return [dxdt, dvdt]

class DataPointInputGUI:
    def __init__(self, master):
        self.master = master
        self.master.title("Data Point Input")

        self.fig, self.ax = plt.subplots()
        self.ax.set_xlabel('X')
        self.ax.set_ylabel('Y')
        self.ax.set_title('X vs Y')
        self.ax.set_xlim(10**8, 10**9)
        self.ax.set_ylim(-1000, 3000)
        self.t_new = []
        self.points = []
        self.line, = self.ax.plot([], [], color='blue')

        self.canvas = FigureCanvasTkAgg(self.fig, master=master)
        self.canvas.draw()
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        self.cid = self.fig.canvas.mpl_connect('button_press_event', self.onclick)

        self.button_clear = tk.Button(master, text="Clear Points", command=self.clear_points)
        self.button_clear.pack(side=tk.BOTTOM)

        self.button_simulate = tk.Button(master, text="Run Simulation", command=self.run_simulation)
        self.button_simulate.pack(side=tk.BOTTOM)

    def onclick(self, event):
        if event.inaxes == self.ax:
            x = event.xdata
            y = event.ydata
            if 10**8 <= x <= 10**9 and -1000<= y <= 3000:
                self.points.append((x, y))
                self.update_plot()
            else:
                tk.messagebox.showwarning("Warning", "Data point must be within the range [1, 2] for both X and Y")

    def update_plot(self):
        self.ax.clear()
        self.ax.set_xlabel('X')
        self.ax.set_ylabel('Y')
        self.ax.set_title('X vs Y')
        self.ax.set_xlim(10**8, 10**9)
        self.ax.set_ylim(-1000, 3000)
        if self.points:
            xs, ys = zip(*self.points)
            #xs, ys = self.interpolate(xs, ys)
            self.line, = self.ax.plot(xs, ys, color='blue')
        self.canvas.draw()

    def interpolate(self, xs, ys):
        xs = np.array(xs)
        ys = np.array(ys)
        xs = np.append(xs, xs[0])  # Closing the curve
        ys = np.append(ys, ys[0])  # Closing the curve
        t = np.linspace(10**8, 10**9, len(xs))
        self.t_new = np.linspace(10**8, 10**9, 10*len(xs))
        f_x = interpolate.interp1d(t, xs, kind='cubic')
        f_y = interpolate.interp1d(t, ys, kind='cubic')
        xs_new = f_x(self.t_new)
        ys_new = f_y(self.t_new)
        return xs_new, ys_new

    def clear_points(self):
        self.points = []
        self.update_plot()
    
    def run_simulation(self):
        y0 = [0.01, 0.01]  # Initial conditions
        Fs = 22050  # sampling frequency
        t = np.arange(0, 0.24, 1/Fs)
        z1 = odeint(model, y0, t, args=(self.points,))
        song = z1[:, 0]
        print(song)
        self.update_plot()


In [ ]:

def main():
    root = tk.Tk()
    app = DataPointInputGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()


In [ ]:
points = [(205241935.48387095, 712.121212121212), (299596774.1935484, 1523.8095238095239), (631653225.8064516, 1740.2597402597403), (684274193.548387, 766.2337662337663), (535483870.9677419, -23.809523809523853), (339516129.03225803, -261.9047619047619), (190725806.4516129, 116.88311688311683), (138104838.70967743, 1015.151515151515), (328629032.2580645, 2032.4675324675327), (442943548.38709676, 1080.08658008658), (283266129.03225803, 593.0735930735932), (156250000.0, 896.1038961038962), (252419354.83870965, 1339.8268398268401), (308669354.83870965, 1004.3290043290044), (261491935.48387098, 701.2987012987014), (239717741.93548387, 733.7662337662337), (223387096.77419353, 712.121212121212)]

y0 = [0.01, 0.01]  # Initial conditions
Fs = 22050  # sampling frequency
t = np.arange(0, 0.24, 1/Fs)
z1 = odeint(model, y0, t, args=(points,))

In [ ]:
import numpy as np
from scipy.integrate import odeint

def singSyllable(rho):
    Fs = 22050  # sampling frequency
    t = np.arange(0, 0.24, 1/Fs)
    
    # Define kt and pt functions
    #kt = 4 * 10**8 * np.cos(4 * np.pi * t + rho) + 9 * 10**8
    #pt = 1005 + 1000 * np.cos(4 * np.pi * t)

    def kt(t,rho):
        return 4 * 10**8 * np.cos(4 * np.pi * t + rho) + 9 * 10**8
    def pt(t,rho):
        return 1005 + 1000 * np.cos(4 * np.pi * t)
    # Solve differential equation
    def syrinx(y, t,rho):
        b = 1000  # (dissipation-friction coefficient)
        d = 10**8  # (nonlinear dissipation coefficient for bounded motion)
        yprime = np.zeros(2)
        yprime[0] = y[1]
        #pt_interp = np.interp(t, t, pt)
        #kt_interp = np.interp(t, t, kt)
        yprime[1] = (pt(t,rho) - b) * y[1] - kt(t,rho) * y[0] - d * (y[0]**2) * y[1]
        return yprime
    
    y0 = [0.01, 0.01]  # Initial conditions
    
    # Ensure t, pt, and kt are 1-dimensional arrays
    t = np.atleast_1d(t)
    #pt = np.atleast_1d(pt)
    #kt = np.atleast_1d(kt)
    
    z1 = odeint(syrinx, y0, t, args=(rho,))
    song = z1[:, 0]
    
    return t, song

t, song = singSyllable(0)


In [ ]:
plt.plot(t,song)

In [ ]:
from scipy.fft import fft, fftfreq
Fs = 22050
s_fft = fft(song)
t_fft = fftfreq(len(t),1/Fs)[0:len(t)//2]

plt.plot(t_fft,2.0/len(t) * np.abs(s_fft[0:len(t)//2]))

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.fft import fft, fftfreq
from ipywidgets import interact

def singSyllable(rho):
    Fs = 22050  # sampling frequency
    t = np.arange(0, 0.24, 1/Fs) #time sample points
    
    #spring constant function
    def kt(t,rho):
        return 4 * 10**8 * np.cos(4 * np.pi * t + rho) + 9 * 10**8
    
    #presure diff function
    def pt(t,rho):
        return 1005 + 1000 * np.cos(4 * np.pi * t)
    
    #main diff equation
    def syrinx(y, t,rho):
        b = 1000
        d = 10**8
        yprime = np.zeros(2)
        yprime[0] = y[1]
        yprime[1] = (pt(t,rho) - b) * y[1] - kt(t,rho) * y[0] - d * (y[0]**2) * y[1]
        return yprime
    
    #init condition
    y0 = [0.01, 0.01]
    
    #ode solver
    z = odeint(syrinx, y0, t, args=(rho,))
    song = z[:, 0]
    
    return t, song

def plot_wave_and_fft(rho):
    t, song = singSyllable(rho)
    
    plt.figure(figsize=(12, 6))
    
    # Plot wave
    plt.subplot(2, 1, 1)
    plt.plot(t, song)
    plt.title('Waveform')
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    
    # Calculate FFT
    Fs = 22050
    s_fft = fft(song)
    t_fft = fftfreq(len(t), 1/Fs)[0:len(t)//2]
    
    # Plot FFT
    plt.subplot(2, 1, 2)
    plt.plot(t_fft, 2.0/len(t) * np.abs(s_fft[0:len(t)//2]))
    plt.title('FFT')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Amplitude')
    
    plt.tight_layout()
    plt.show()

# Add slider for rho
interact(plot_wave_and_fft, rho=(-np.pi, np.pi, 0.01))


interactive(children=(FloatSlider(value=-0.0015926535897929917, description='rho', max=3.141592653589793, min=…

<function __main__.plot_wave_and_fft(rho)>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.fft import fft, fftfreq

def singSyllable(K, P):
    Fs = 22050  # sampling frequency
    t = np.arange(0, 0.24, 1/Fs)  # time sample points

    # main diff equation
    def syrinx(y, t, K, P):
        b = 1000
        d = 10**8
        yprime = np.zeros(2)
        yprime[0] = y[1]
        yprime[1] = (P - b) * y[1] - K * y[0] - d * (y[0]**2) * y[1]
        return yprime

    # init condition
    y0 = [0.01, 0.01]

    # ode solver
    z = odeint(syrinx, y0, t, args=(K, P,))
    song = z[:, 0]

    return t, song

def plot_wave_and_fft(K, P):
    t, song = singSyllable(K, P)

    plt.figure(figsize=(12, 6))

    # Plot wave
    plt.subplot(2, 1, 1)
    plt.plot(t, song)
    plt.title('Waveform')
    plt.xlabel('Time')
    plt.ylabel('Amplitude')

    # Calculate FFT
    Fs = 22050
    s_fft = fft(song)
    t_fft = fftfreq(len(t), 1/Fs)[0:len(t)//2]

    # Plot FFT
    plt.subplot(2, 1, 2)
    plt.plot(t_fft, 2.0/len(t) * np.abs(s_fft[0:len(t)//2]))
    plt.title('FFT')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Amplitude')

    plt.tight_layout()
    plt.show()

def onclick(event):
    if event.inaxes is not None:
        K = event.xdata
        P = event.ydata
        plot_wave_and_fft(K, P)

# Create a grid of K and P values
K_values = np.linspace(10**8, 10**9, 100)
P_values = np.linspace(-1000, 3000, 100)
K_mesh, P_mesh = np.meshgrid(K_values, P_values)

# Plot the contour map
plt.figure(figsize=(8, 6))
plt.imshow(np.zeros_like(K_mesh), extent=[K_values.min(), K_values.max(), P_values.min(), P_values.max()],
           aspect='auto', cmap='viridis', origin='lower', alpha=0.5)
plt.xlabel('K')
plt.ylabel('P')
plt.title('Click on the contour to set K and P')

plt.gca().figure.canvas.mpl_connect('button_press_event', onclick)

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.fft import fft, fftfreq

def singSyllable(K, P):
    Fs = 22050  # sampling frequency
    t = np.arange(0, 0.24, 1/Fs)  # time sample points

    # main diff equation
    def syrinx(y, t, K, P):
        b = 1000
        d = 10**8
        yprime = np.zeros(2)
        yprime[0] = y[1]
        yprime[1] = (P - b) * y[1] - K * y[0] - d * (y[0]**2) * y[1]
        return yprime

    # init condition
    y0 = [0.01, 0.01]

    # ode solver
    z = odeint(syrinx, y0, t, args=(K, P,))
    song = z[:, 0]

    return t, song

def plot_wave_and_fft(K, P):
    t, song = singSyllable(K, P)

    plt.figure(figsize=(12, 6))

    # Plot wave
    plt.subplot(2, 1, 1)
    plt.plot(t, song)
    plt.title('Waveform')
    plt.xlabel('Time')
    plt.ylabel('Amplitude')

    # Calculate FFT
    Fs = 22050
    s_fft = fft(song)
    t_fft = fftfreq(len(t), 1/Fs)[0:len(t)//2]

    # Plot FFT
    plt.subplot(2, 1, 2)
    plt.plot(t_fft, 2.0/len(t) * np.abs(s_fft[0:len(t)//2]))
    plt.title('FFT')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Amplitude')

    plt.tight_layout()
    plt.show()

def select_K_P():
    plt.figure(figsize=(8, 6))
    plt.xlabel('K')
    plt.ylabel('P')
    plt.title('Click on the contour to set K and P')

    K, P = plt.ginput(1)[0]
    plot_wave_and_fft(K, P)

select_K_P()
